In [ ]:
# Load gemini
from context import gemini
from gemini  import data, engine, helpers

# Global Imports
import pandas as pd
import numpy as np

# Build mean reversion strategy
from talib.abstract import *

In [ ]:
def bands(df, timeperiod=26, nbdevup=2.6, nbdevdn=2.6, matype=0):
    cols = ['high', 'low', 'open', 'close', 'volume']
    HLOCV = {key: df[key].values for key in df if key in cols}
    u, m, l = BBANDS(HLOCV, timeperiod=timeperiod, nbdevup=nbdevup, nbdevdn=nbdevdn, matype=matype)
    df['upper']  = u
    df['middle'] = m
    df['lower']  = l
    return df

def touches(df):
    df['touch_upper'] = df.high >= df.upper
    df['touch_lower'] = df.low  <= df.lower
    df['crossing_dn'] = (df.close < df.middle) & (df.open > df.middle)
    df['crossing_up'] = (df.close > df.middle) & (df.open < df.middle)
    return df

In [10]:
def logic(account, lookback):
    try:
        lookback = helpers.period(lookback)
        today = lookback.loc(0)
        
        # Selling
        if today.touch_upper:
            exit_price = today.upper
            for position in account.positions:  
                if position.type == 'long':
                    account.close_position(position=position, 
                                           percent=1.0, 
                                           current_price=exit_price)

        if today.crossing_up:
            exit_price = today.close
            for position in account.positions:  
                if position.type == 'long':
                    account.close_position(position=position, 
                                           percent=1.0, 
                                           current_price=exit_price)
                    
        # Buying
        if today.touch_lower | today.crossing_dn:
            risk          = 1
            entry_price   = today.lower
            entry_capital = account.buying_power*risk
            if entry_capital > 0:
                account.enter_position(type='long', 
                                       entry_capital=entry_capital, 
                                       entry_price=entry_price)
        if today.crossing_dn:
            risk          = 1
            entry_price   = today.close
            entry_capital = account.buying_power*risk
            if entry_capital > 0:
                account.enter_position(type='long', 
                                       entry_capital=entry_capital, 
                                       entry_price=entry_price)
    
    except Exception as e:
        print(e)
        pass # Handles lookback errors in beginning of dataset

In [11]:
# Apply strategy to example
df = pd.read_csv("data/BTC_USD.csv", header=0, index_col=0)
df['date'] = pd.to_datetime(df['date'])
df.head()

,date,low,high,open,close,volume
0,2017-01-16,816.00000000,834.15680536,818.76200000,829.00000001,643792.21530254
1,2017-01-17,827.75970935,908.53017772,829.00000001,902.70000066,2679383.29369767
2,2017-01-18,850.00000000,912.00000000,902.27579339,886.20000033,1959863.34974819
3,2017-01-19,883.30000000,906.99999900,886.20000054,900.23240036,1219112.38795964
4,2017-01-20,882.54320000,902.42999999,900.30000001,895.00000000,841765.13205928


In [12]:
df = bands(df)
df = touches(df)

In [13]:
backtest = engine.backtest(df)

In [14]:
output = backtest.start(1000, logic)
output.tail()

,benchmark_equity,strategy_equity,benchmark_return,strategy_return
date,,,,
2017-11-27,9711.00000013,6865.28571855,0.04595519,0.00000000
2017-11-28,9879.96875865,6865.28571855,0.01739973,0.00000000
2017-11-29,9702.00000000,6865.28571855,-0.01801309,0.00000000
2017-11-30,9907.67682035,6865.28571855,0.02119942,0.00000000
2017-12-01,9659.09950232,6865.28571855,-0.02508936,0.00000000


In [15]:
backtest.results()

-------------- Results ----------------

Buy and Hold : 1079.72%
Net Profit   : 10797.2
Strategy     : 586.53%
Net Profit   : 5865.29
Longs        : 9
sells        : 10
shorts       : 0
covers       : 0
--------------------
Total Trades : 19

---------------------------------------


### Feed results into Pyfolio analysis

In [9]:
import pyfolio as pf
%matplotlib inline

# silence warnings
import warnings
warnings.filterwarnings('ignore')

/Users/anthonyfederico/anaconda3/lib/python3.6/site-packages/theano/configdefaults.py:560: UserWarning: DeprecationWarning: there is no c++ compiler.This is deprecated and with Theano 0.11 a c++ compiler will be mandatory
  warnings.warn("DeprecationWarning: there is no c++ compiler."
WARNING (theano.configdefaults): g++ not detected ! Theano will be unable to execute optimized C-implementations (for both CPU and GPU) and will default to Python implementations. Performance will be severely degraded. To remove this warning, set Theano flags cxx to an empty string.
WARNING (theano.tensor.blas): Using NumPy C-API based implementation for BLAS functions.
/Users/anthonyfederico/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [ ]:
pf.create_returns_tear_sheet(output['strategy_return'], benchmark_rets=output['benchmark_return'])